# Geometry Optimization

When we perform an electronic structure calculation the most important input is the crystal structure. The results are entirely dependent upon the structure that you have used, i.e., what are the atomic positions, the lattice parameters, angles, etc. That is why the first step of any calculation is to optimize the crystal structure.

In this notebook, we will optimize the in-plane lattice parameter for graphene structure. While most commercial DFT codes have built in optimization routines, we are going to manually optimize our structure. We only have a single parameter to optimize so this is doable by hand (and a little bit of code!).

**How are we going to optimize the a lattice parameter?**

In essence, we are going to perform an experiment albeit a computational one. 

- adjust lattice parameter
- calculate energy
- repeat
- plot the results


The minimum of this curve will tell us the optimal lattice parameter. The minimum of this curve is the optimal lattice parameter, because nature is always trying to minimize it's total energy, or in other words, nature is lazy!

In [1]:
# load dependencies
using DFTK
using Plots

## Crystal Structure of Graphene
In the image below, we have the crystal structure of graphene. The blue spheres denote carbon atoms that are arranged in a 2d hexagonal lattice, where every carbon atoms has three nearest neighbors. The black border denotes the unit cell of the crystal.

<img src="../../structure/graphene_img.png" alt="graphene" width="300" align="left"/>

In [2]:
# Compute the energy
# For now we haven't covered the details of how this function is working. This will be
# done tomorrow. We will simple use this for now. All we need to know is that it is a 
# function that takes as input the the lattice parameter a and outputs the energy.

function compute_scf(a; c = 10,  
                        positions = [[1/3., 2/3., 0], [2/3., 1/3., 0]],
                        Ecut=15, kgrid=[20, 20, 2],
                        tol=1e-8,
                        convert2bohr = x -> x/0.53)
    a = convert2bohr(a)
    c = convert2bohr(c)
    C = ElementPsp(:C, psp=load_psp("hgh/lda/c-q4.hgh"))
    atoms = [C => [pos for pos in positions]]
    lattice = [[a -0.5*a 0]; [0 0.5*sqrt(3)*a 0]; [0 0 c]]
    model = model_LDA(lattice, atoms, temperature = 0.01)
    basis = PlaneWaveBasis(model, Ecut; kgrid=kgrid)
    scfres = self_consistent_field(basis; tol=tol, callback=info->nothing)
    total_E = scfres.energies.total;
    return total_E
end;

In [ ]:
# Write a function that computes the energy for a list of lattice parameters.
function calcEnergies(trials)

end;

In [3]:
# Write a function that computes the energy for a list of lattice parameters.
# Optional: Add a print statement so that once the energy is computed it will print the
#           lattice parameter used and the energy at this data.
# Note! Make sure that you store the energies in an array.

function calcEnergies(trial)
    energies = []
    println("|   iter    |     a     |     Energy       |")
    println("|------------------------------------------|")
    for (i, a) in enumerate(trial)
        E = compute_scf(a)
        append!(energies, E)
    println("| $i          $(round(a, sigdigits=4))  $E |")
    end
    println("|------------------------------------------|")
    return energies
end;

In [ ]:
# run code

In [ ]:
# plot the results

In [ ]:
# Example of what output will look like.
trial_a = convert(Array, LinRange(2., 3., 10))
energies = calcEnergies(trial_a);

|   iter    |     a     |     Energy       |
|------------------------------------------|
| 1          2.0  -10.82582735544047 |
| 2          2.111  -11.001348445099254 |
| 3          2.222  -11.108411936513262 |


In [ ]:
plot(trial_a, energies, legend=false, m=:circle, 
    markersize=3, color=1, linestyle=:solid, 
    markerstrokecolor=1)
plot!(title = "Geometry Optimization", xlabel = "a (ang.)", ylabel = "Energy (Ha)")
plot!(xlims = (0, 5.5), ylims = (-2.2, 6), xticks = 0:0.5:10, yticks = [0,1,5,10])

We can see the minimum of our energy versus displacement curve occurs around 2.46 ang., which is the correct in-plane lattice parameter for graphene.